In [17]:
%pwd

'e:\\LLM-Powered-Medical-Knowledge-Chatbot-using-RAG-and-AWS'

In [18]:
import os
os.chdir("../")

In [19]:
%pwd

'e:\\'

In [20]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

import sentence_transformers

import langchain

In [21]:
!pip install -q langchain

print(f'Langchain Version: {langchain.__version__}')
import sys
print(f'Python Version: {sys.version}')

Langchain Version: 1.2.8
Python Version: 3.10.6 (tags/v3.10.6:9c7b4bd, Aug  1 2022, 21:53:49) [MSC v.1932 64 bit (AMD64)]


In [22]:
# Extract the Data From PDF file

def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob= "*pdf",
                             loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents

In [23]:
extracted_data= load_pdf_file(data='Data/')

In [24]:
# extracted_data

In [25]:
# Split the Data into the Text Chunks

def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks= text_splitter.split_documents(extracted_data)
    return text_chunks

In [26]:
text_chunks = text_split(extracted_data)
print("Length of Text chunks", len(text_chunks))

Length of Text chunks 0


In [27]:
def download_hugging_face_embeddings():
    return HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-V2")

In [28]:

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-V2")


In [29]:
query_result = embeddings.embed_query("hello world")
print("Length", len(query_result))

Length 384


In [30]:
# query_result